## Event based

In [2]:
from cartopy import config
import cartopy
import cartopy.crs as ccrs
import climtas
import dask.array
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from scipy import stats
from scipy.stats import ttest_ind 
import seaborn as sns
import sparse
import xarray as xr

In [83]:
lat = -23.25
lon = 113.5

In [124]:
# opening the input files with heatwave severity data 
thw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-coastal-sev.nc').sel(latitude=lat, longitude=lon)
mhw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-ocean-sev-2.nc').sel(latitude=lat, longitude=lon)

In [125]:
# using climtas.event.find_events to identify days where severity>1 for a minimum of 3 consecutive days 
thw_events = climtas.event.find_events(thw.severity > 1, min_duration = 3)

In [126]:
# Create a sparse array with (coords, values, shape)
event_duration_array = sparse.COO([thw_events['time']], thw_events['event_duration'], shape=thw.severity.shape)
event_duration_da = xr.DataArray(event_duration_array, coords=thw.severity.coords)

In [127]:
def get_coords(da, events):
    # Convert the index values to coordinates
    coords = {}
    for d in da.dims:
        coords[d] = da[d].values[events[d].values]
    
    # Also work out when the event ends
    coords['time_end'] = da['time'].values[events['time'].values + events['event_duration'].values-1]
    #coords['event_duration'] = coords['time_end'] - coords['time'] 
    coords['event_duration'] = events['event_duration'].values
    
    return pd.DataFrame(coords, index=events.index)

In [128]:
thw_new_H = get_coords(event_duration_da, thw_events)
thw_new_H = thw_new_H.rename(columns={"time" : "time_thw", "time_end" : "time_end_thw", "event_duration": "event_duration_thw"})
thw_new_H

,time_thw,time_end_thw,event_duration_thw
0,1982-12-06,1982-12-08,3
1,1983-06-07,1983-06-11,5
2,1983-08-07,1983-08-10,4
3,1983-09-16,1983-09-21,6
4,1983-10-22,1983-10-25,4
...,...,...,...
166,2018-01-02,2018-01-06,5
167,2018-01-12,2018-01-14,3
168,2018-03-07,2018-03-11,5
169,2018-05-14,2018-05-16,3


In [129]:
# using climtas.event.find_events to identify days where severity>1 for a minimum of 5 consecutive days (MHW events)
mhw_events = climtas.event.find_events(mhw.severity > 1, min_duration = 5)

In [130]:
mhw_duration_array = sparse.COO([mhw_events['time']], mhw_events['event_duration'], shape=mhw.severity.shape)
mhw_duration_da = xr.DataArray(event_duration_array, coords=mhw.severity.coords)

In [131]:
mhw_new = get_coords(mhw_duration_da, mhw_events)
mhw_new

,time,time_end,event_duration
0,1982-03-15,1982-03-20,6
1,1982-04-16,1982-04-20,5
2,1982-04-28,1982-05-10,13
3,1982-08-07,1982-08-11,5
4,1982-08-15,1982-08-19,5
...,...,...,...
96,2015-09-15,2015-10-17,33
97,2015-10-21,2015-10-30,10
98,2016-07-17,2016-07-21,5
99,2017-02-28,2017-03-06,7


In [132]:
df_concat_col = pd.concat([thw_new_H, mhw_new], axis=1, sort=False)
df_concat_col

,time_thw,time_end_thw,event_duration_thw,time,time_end,event_duration
0,1982-12-06,1982-12-08,3,1982-03-15,1982-03-20,6.0
1,1983-06-07,1983-06-11,5,1982-04-16,1982-04-20,5.0
2,1983-08-07,1983-08-10,4,1982-04-28,1982-05-10,13.0
3,1983-09-16,1983-09-21,6,1982-08-07,1982-08-11,5.0
4,1983-10-22,1983-10-25,4,1982-08-15,1982-08-19,5.0
...,...,...,...,...,...,...
166,2018-01-02,2018-01-06,5,NaT,NaT,NaN
167,2018-01-12,2018-01-14,3,NaT,NaT,NaN
168,2018-03-07,2018-03-11,5,NaT,NaT,NaN
169,2018-05-14,2018-05-16,3,NaT,NaT,NaN


In [133]:
#Calculating the overlap between THW and MHW events 
from datetime import datetime
from collections import namedtuple
Range = namedtuple('Range', ['start', 'end'])
overlap = []
for n in range (df_concat_col.shape[0]):
    mhw = Range(start=df_merge_col['time'].iloc[n], end=df_merge_col['time_end'].iloc[n])
    thw = Range(start=df_merge_col['time_thw'].iloc[n], end=df_merge_col['time_end_thw'].iloc[n])
    latest_start = max(mhw.start, thw.start)
    earliest_end = min(mhw.end, thw.end)
    delta = (earliest_end - latest_start).days + 1
    b = max(0, delta)
    overlap.append(b)

In [152]:
mod_fd = df_merge_col.assign(overlap_days = overlap)
mod_fd

,time_thw,time_end_thw,event_duration,time,time_end,event_duration,overlap_days
0,1982-12-06,1982-12-08,3,1982-03-15,1982-03-20,6.0,0
1,1983-06-07,1983-06-11,5,1982-04-16,1982-04-20,5.0,0
2,1983-08-07,1983-08-10,4,1982-04-28,1982-05-10,13.0,0
3,1983-09-16,1983-09-21,6,1982-08-07,1982-08-11,5.0,0
4,1983-10-22,1983-10-25,4,1982-08-15,1982-08-19,5.0,0
...,...,...,...,...,...,...,...
166,2018-01-02,2018-01-06,5,NaT,NaT,NaN,0
167,2018-01-12,2018-01-14,3,NaT,NaT,NaN,0
168,2018-03-07,2018-03-11,5,NaT,NaT,NaN,0
169,2018-05-14,2018-05-16,3,NaT,NaT,NaN,0


In [153]:
del mod_fd['event_duration']
del mod_fd['time']
del mod_fd['time_end']

mod_fd

,time_thw,time_end_thw,overlap_days
0,1982-12-06,1982-12-08,0
1,1983-06-07,1983-06-11,0
2,1983-08-07,1983-08-10,0
3,1983-09-16,1983-09-21,0
4,1983-10-22,1983-10-25,0
...,...,...,...
166,2018-01-02,2018-01-06,0
167,2018-01-12,2018-01-14,0
168,2018-03-07,2018-03-11,0
169,2018-05-14,2018-05-16,0


In [145]:
thw_only = mod_fd[mod_fd.overlap_days == 0]#.reset_index()

del thw_only['event_duration']
del thw_only['time']
del thw_only['time_end']
del thw_only['overlap_days']
thw_only

,time_thw,time_end_thw
0,1982-12-06,1982-12-08
1,1983-06-07,1983-06-11
2,1983-08-07,1983-08-10
3,1983-09-16,1983-09-21
4,1983-10-22,1983-10-25
...,...,...
166,2018-01-02,2018-01-06
167,2018-01-12,2018-01-14
168,2018-03-07,2018-03-11
169,2018-05-14,2018-05-16


In [154]:
mod_fd_ds = mod_fd.to_xarray()

In [116]:
# MSLP absolute values 

ds = xr.open_mfdataset('/g/data/e14/cp3790/Charuni/ERA5-MSL/era5_dailymsl_*.nc').sel(longitude=slice(90, 180), latitude=slice(0, -60))
mslp = ds.dmsl

In [158]:
composite = mslp.where(mod_fd_ds['overlap_days']==0, drop=True)

ValueError: dimensions {'index'} do not exist. Expected one or more of ('time', 'latitude', 'longitude')